# 陳時中、黃珊珊、蔣萬安市長候選人大PK

Penalty Kick or Player Kill for Taipei city mayor candidates--Chen, Huang, and Jiang


What is share of voice? 品牌聲量

        Share of voice (SOV) is a measure of the market your brand owns compared to your competitors. It acts as a gauge for your brand visibility and how much you dominate the conversation in your industry. The more market share you have, the greater popularity and authority you likely have among users and prospective customers.

什麼是SOV？

        SOV又稱Share of Vocie，是衡量一切有關品牌聲量的指標，在數位時代，SOV包含品牌在特定關鍵字、搜尋引擎以及社群聲量的占比，是衡量多少消費者認知到您品牌的重要指標。
        
[來源](https://digitalpr.tw/%E7%A4%BE%E7%BE%A4%E5%8F%A3%E7%A2%91/sov%E7%A4%BE%E7%BE%A4%E8%81%B2%E9%87%8F%E5%81%A5%E6%AA%A2/)


# Put them together

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import re

# Load Data 
df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')
# Step 0: Filter news articles using the following function
# Searching keywords from "content" column
def filter_df_via_content(df, query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query

# Step 1: Sentimental polarity score計算整體情緒分數(影響力)
# sentimental polarity score
def get_article_sentiment(df_query):
    # df_query = df[df['tokens'].str.contains(query_key)]
    sentiCount = {'pos': 0, 'neg': 0, 'obj': 0}
    sentiPercnt = {'pos': 0, 'neg': 0, 'obj': 0}
    numberOfArticle = len(df_query)
    for senti in df_query.sentiment:
        # 判斷文章的情緒極性
        if senti >= 0.75:
            sentiCount['pos'] += 1
        elif senti <= 0.4:
            sentiCount['neg'] += 1
        else:
            sentiCount['obj'] += 1
    for polar in sentiCount :
        sentiPercnt[polar]=round(sentiCount[polar]/numberOfArticle*100)

    return sentiCount, sentiPercnt

# **計算各類別多少篇文章提到該關鍵字
# **計算各類別出現關鍵字次數

news_categories = ['政治', '科技', '運動', '證卷', '產經', '娛樂', '生活', '國際', '社會', '文化', '兩岸', '全部']



def count_keyword(df_query, query_keywords):

    cate_occurrence = {}
    cate_freq = {}
    
    # 字典初始化
    for cate in news_categories:
        cate_occurrence[cate] = 0   # {'政治':0, '科技':0}
        cate_freq[cate] = 0

    for idx, row in df_query.iterrows():
        # count the number of articles各類別篇數統計
        cate_occurrence[row.category] += 1  #   {'政治':+1, '科技':0}
        cate_occurrence['全部'] += 1
        
        # count the keyword frequency各類別次數統計
        # 計算這一篇文章的content中重複含有多少個這些關鍵字(頻率)
        freq = sum([ len(re.findall(keyword, row.content, re.I)) for keyword in query_keywords]) 
        cate_freq[row.category] += freq # 在該新聞類別中累計頻率
        cate_freq['全部'] += freq  # 在"全部"類別中累計頻率

    total_articles = cate_occurrence['全部']  # len(df_query)
    total_frequency = cate_freq['全部']
    return cate_freq, cate_occurrence, total_articles, total_frequency

# 與上一週的程式碼有些不一樣，讓每個PK的對象之資料的start_date end_date都相同，折線圖才會完整
def get_keyword_occurrence_time_series(df_query):


    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)
    

    # 設定時間欄位'date_index', 次數freq欄位，將每一篇的時間寫到'date_index'欄位，將freq設定為1 ==> 進行groupby之後就可以合併日期與加總freq次數
    query_freq = pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'freq':[1 for _ in range(len(df_query))]})

    # 開始時間、結束時間兩項必須也加入到query_freq，計算次數時才會有完整的時間軸，否則時間軸長度會因為新聞時間不同，導致時間軸不一致
    dt_start_date = datetime.strptime(start_date, '%Y-%m-%d')
    dt_end_date = datetime.strptime(end_date, '%Y-%m-%d')   

    query_freq = pd.concat([query_freq, pd.DataFrame({'date_index': [dt_start_date], 'freq': [0]})])
    query_freq = pd.concat([query_freq, pd.DataFrame({'date_index': [dt_end_date], 'freq': [0]})])
    # query_freq = query_freq.append({'date_index': dt_end_date, 'freq': 0}, ignore_index=True)

    # 透過groupby就可以合併日期與加總freq次數
    freq_data_group = query_freq.groupby(pd.Grouper(key='date_index', freq='D')).sum()

    # 'date_index'為index(索引)，將其變成欄位名稱，inplace=True表示原始檔案會被異動
    freq_data_group.reset_index(inplace=True)
    # freq_data_group = freq_data_group.reset_index() # 這樣也可以得到同樣結果

    # 只有頻率次數值y, 沒有時間變數x, 計算相關係數時會用到
    # freq_data = freq_data_group.freq.to_list()

    # 有時間變數x,y，用於畫趨勢線圖
    line_xy_data = [{'x':date.strftime('%Y-%m-%d'),'y':freq} for date, freq in zip(freq_data_group.date_index,freq_data_group.freq)]

    return line_xy_data

# Step 4: Process PK data
# 定義PK對象
# 通常會多個關鍵字代表該PK對象，['柯文哲','柯p','柯P']
list_pkNames = ['陳時中', '黃珊珊', '蔣萬安']
list_pkKeywordSet = [['陳時中'], ['黃珊珊'], ['蔣萬安']]

# list_pkKeywordSet = [['陳時中'], ['黃珊珊'], ['蔣萬安'], ['柯文哲','柯p','柯P']]


# 線條顏色
list_colors = ["rgba(0,128,0,0.2)",'rgba(0,255,255,0.2)','rgba(0,0,255,0.2)']
#list_colors = ['green', 'red', 'blue']

# 人頭圖案
list_photos = [
    "https://upload.wikimedia.org/wikipedia/commons/c/c0/%E8%A1%9B%E7%94%9F%E7%A6%8F%E5%88%A9%E9%83%A8%E9%83%A8%E9%95%B7%E9%99%B3%E6%99%82%E4%B8%AD.jpg",
    "https://councilorwatch.tw/images/councillors/female/230.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/9/93/%E8%94%A3%E8%90%AC%E5%AE%89%E5%A7%94%E5%93%A1.jpg"
]

# 準備長條圖 線圖 總篇數 總次數等數據
list_freq_daily_line_chart = []
list_freq_news_category = []
list_sentimentInfo = []

list_total_articles=[]
list_total_frequency=[]

weeks = 12
cate = "全部"
cond = 'or'

selectedCategories = ['全部', '政治', '兩岸', '產經', '生活', '社會']
# selectedCategories = ['全部','政治', '產經', '生活', '國際', '社會', '兩岸']

for query_keywords in list_pkKeywordSet:

    # Filter news   
    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
    #df_query = filter_df_via_tokens(df, query_keywords, cond, cate, weeks)
    
    # Get line chart data
    line_xy_data = get_keyword_occurrence_time_series(df_query)
    list_freq_daily_line_chart.append(line_xy_data)
    
    # Get bar chart data, total articals and frequecy 
    cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(df_query, query_keywords)
    list_total_articles.append(total_articles)
    list_total_frequency.append(total_frequency)


    # We select these categories to display: ['全部', '政治', '兩岸', '產經', '生活', '社會']
    cate_freq_selected = [cate_occurrence[cate] for cate in selectedCategories]
    list_freq_news_category.append(cate_freq_selected)

    # Get sentiment information
    sentiCount, sentiPercnt = get_article_sentiment(df_query)
    #sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
    #                                            str(sentiPercnt['neg']))
    senti_info = [sentiPercnt[p] for p in ['pos','obj','neg']] # 只要取用pos,obj, neg的情緒數字篇數百分比
    list_sentimentInfo.append(senti_info)

# We need all the following data to display on our frontend page 有一大堆數據要送到前端去展示啊
pk_data =  {'list_freq_daily_line_chart': list_freq_daily_line_chart,
           'list_pkNames': list_pkNames,
           'list_colors': list_colors,
           'list_photos': list_photos,
           'list_freq_news_category': list_freq_news_category,
           'list_category': selectedCategories,
           'list_sentiInfo': list_sentimentInfo,
           'list_total_articles': list_total_articles,
           'list_total_frequency': list_total_frequency
           }

# Step 5: Save PK data to csv file
df_data_pk = pd.DataFrame(list(pk_data.items()),columns=['name','value'])
## 存成csv格式檔案
df_data_pk.to_csv('pk_taipei_mayor_election.csv',sep=',', index=None)

# Load Data 

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import re

In [3]:

df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')

In [4]:
df.head(1)

,item_id,title,category,content,link,date,photo_link,tokens_v2,top_key_freq,summary,sentiment
0,aipl_20220124_1001,殲16D首擾台 專家示警：國軍應強化電子作戰,政治,中共殲16D新型電戰機今天首次擾台，學者及退將分析，殲16D可能已量產並進行實戰化運用，國軍...,https://www.cna.com.tw/news/aipl/202201240357....,2022-01-24,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...,"['中共', '新型', '電戰機', '學者', '退將', '分析', '量產', '進...","[('中共', 7), ('台灣', 7), ('電子', 6), ('美軍', 6), (...","['其中殲16D新型電戰機為首次現蹤', '中共殲16D新型電戰機今天首次擾台', '13架...",0.96


In [5]:
df.shape

(26179, 11)

# Step 0: Filter news articles using the following function

In [6]:
from datetime import datetime, timedelta

In [7]:
from datetime import datetime, timedelta
# Searching keywords from "content" column
# Here this function uses df.content column, while filter_dataFrame() uses df.tokens_v2
def filter_df_via_content(df, query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


# Step 1: Sentimental polarity score計算整體情緒分數(影響力)

In [8]:
# sentimental polarity score
def get_article_sentiment(df_query):
    # df_query = df[df['tokens'].str.contains(query_key)]
    sentiCount = {'pos': 0, 'neg': 0, 'obj': 0}
    sentiPercnt = {'pos': 0, 'neg': 0, 'obj': 0}
    numberOfArticle = len(df_query)
    for senti in df_query.sentiment:
        # 判斷文章的情緒極性
        if senti >= 0.75:
            sentiCount['pos'] += 1
        elif senti <= 0.4:
            sentiCount['neg'] += 1
        else:
            sentiCount['obj'] += 1
    for polar in sentiCount :
        sentiPercnt[polar]=round(sentiCount[polar]/numberOfArticle*100)
        #sentiPercnt[polar]=round(sentiCount[polar]/numberOfArticle,2)
    return sentiCount, sentiPercnt


In [9]:
query_keywords = ['黃珊珊']
weeks=12
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
sentiCount, sentiPercnt = get_article_sentiment(df_query)

In [10]:
sentiCount

{'pos': 65, 'neg': 34, 'obj': 34}

In [11]:
sentiPercnt

{'pos': 49, 'neg': 26, 'obj': 26}

In [12]:
# sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
#                                            str(sentiPercnt['neg']))
[sentiPercnt[p] for p in ['pos', 'obj', 'neg']]

[49, 26, 26]

# Step 2: Count categorical frequency and occurrence(number of articles)

    Count how many articles containing these keywords計算各類別多少篇文章提到該關鍵字
    Count how many times these keywords were mentioned in each category計算各類別出現關鍵字次數

In [13]:
# **計算各類別多少篇文章提到該關鍵字
# **計算各類別出現關鍵字次數

news_categories = ['政治', '科技', '運動', '證卷', '產經', '娛樂', '生活', '國際', '社會', '文化', '兩岸', '全部']


def count_keyword(df_query, query_keywords):

    cate_occurrence = {} # number of articles
    cate_freq = {} # frequency of keyword
    
    # 字典初始化
    for cate in news_categories:
        cate_occurrence[cate] = 0   # {'政治':0, '科技':0}
        cate_freq[cate] = 0

    for idx, row in df_query.iterrows():
        # count the number of articles各類別篇數統計
        cate_occurrence[row.category] += 1  #   {'政治':+1, '科技':0}
        cate_occurrence['全部'] += 1
        
        # count the keyword frequency各類別次數統計
        # 計算這一篇文章的content中重複含有多少個這些關鍵字(頻率)
        freq = sum([ len(re.findall(keyword, row.content, re.I)) for keyword in query_keywords]) 
        cate_freq[row.category] += freq # 在該新聞類別中累計頻率
        cate_freq['全部'] += freq  # 在"全部"類別中累計頻率

    total_articles = cate_occurrence['全部']  # len(df_query)
    total_frequency = cate_freq['全部']


    
    return cate_freq, cate_occurrence, total_articles, total_frequency

In [14]:
query_keywords = ['黃珊珊']
#query_keywords = ['陳時中']
weeks=12
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword(df_query, query_keywords)

({'政治': 132,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 2,
  '娛樂': 12,
  '生活': 236,
  '國際': 0,
  '社會': 2,
  '文化': 0,
  '兩岸': 0,
  '全部': 384},
 {'政治': 44,
  '科技': 0,
  '運動': 0,
  '證卷': 0,
  '產經': 2,
  '娛樂': 5,
  '生活': 80,
  '國際': 0,
  '社會': 2,
  '文化': 0,
  '兩岸': 0,
  '全部': 133},
 133,
 384)

In [15]:
query_keywords = ['黃珊珊']
#query_keywords = ['陳時中']
weeks=12
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword_via_tokens_v2(df_query, query_keywords)

NameError: name 'count_keyword_via_tokens_v2' is not defined

In [ ]:
df_query.head(1)

,item_id,title,category,content,link,date,photo_link,tokens_v2,top_key_freq,summary,sentiment
1822,ahel_20220129_1035,北市幼兒園師案無新增 確診童應是被老師傳染,生活,台北市副市長黃珊珊今天表示，幼兒園確診老師案目前沒有新增確診案例，但確診的孩童Ct值開始下降...,https://www.cna.com.tw/news/ahel/202201290064....,2022-01-29,https://imgcdn.cna.com.tw/www/webphotos/WebCov...,"['台北市', '副市長', '黃珊珊', '幼兒園', '老師', '案例', '確診',...","[('柯文哲', 6), ('黃珊珊', 4), ('老師', 4), ('北市', 4),...","['幼兒園確診老師案目前沒有新增確診案例', '台北市長柯文哲、副市長黃珊珊今早至大佳河濱公...",0.41


### Count word frequency using regular expression

In [ ]:
content = "台北市副市長黃珊珊今天表示，幼兒園確診老師案目前沒有新增確診案例，但確診的孩童Ct值開始下降，黃珊珊...黃珊珊黃珊珊..."
keyword = "黃珊珊"
re.findall(keyword, content, re.I)  # re.I ignors case

['黃珊珊', '黃珊珊', '黃珊珊', '黃珊珊']

In [ ]:
len(re.findall(keyword, content, re.I))

4

In [ ]:
query_keywords = ["黃珊珊", "確診"]
content = "台北市副市長黃珊珊今天表示，幼兒園確診老師案目前沒有新增確診案例，但確診的孩童Ct值開始下降，黃珊珊...黃珊珊黃珊珊..." 
[len(re.findall(keyword, content, re.I)) for keyword in query_keywords]

[4, 3]

In [ ]:
sum([len(re.findall(keyword, content, re.I)) for keyword in query_keywords])

7

# Step 3: Caclulate daily-basis number of articles which contain the query keywords

計算以時間為基礎的被報導的篇數

In [ ]:
def get_keyword_occurrence_time_series(df_query):


    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)
    

    # 設定時間欄位'date_index', 次數freq欄位，將每一篇的時間寫到'date_index'欄位，將freq設定為1 ==> 進行groupby之後就可以合併日期與加總freq次數
    query_freq = pd.DataFrame({'date_index':pd.to_datetime( df_query.date ),'freq':[1 for _ in range(len(df_query))]})

    # 開始時間、結束時間兩項必須也加入到query_freq，計算次數時才會有完整的時間軸，否則時間軸長度會因為新聞時間不同，導致時間軸不一致
    dt_start_date = datetime.strptime(start_date, '%Y-%m-%d')
    dt_end_date = datetime.strptime(end_date, '%Y-%m-%d')   

    query_freq = pd.concat([query_freq, pd.DataFrame({'date_index': [dt_start_date], 'freq': [0]})])
    query_freq = pd.concat([query_freq, pd.DataFrame({'date_index': [dt_end_date], 'freq': [0]})])

    # 透過groupby就可以合併日期與加總freq次數
    freq_data_group = query_freq.groupby(pd.Grouper(key='date_index', freq='D')).sum()

    # 'date_index'為index(索引)，將其變成欄位名稱，inplace=True表示原始檔案會被異動
    freq_data_group.reset_index(inplace=True)
    # freq_data_group = freq_data_group.reset_index() # 這樣也可以得到同樣結果

    # 只有頻率次數值y, 沒有時間變數x, 計算相關係數時會用到
    # freq_data = freq_data_group.freq.to_list()

    # 有時間變數x,y，用於畫趨勢線圖
    line_xy_data = [{'x':date.strftime('%Y-%m-%d'),'y':freq} for date, freq in zip(freq_data_group.date_index,freq_data_group.freq)]

    return line_xy_data


In [ ]:
query_keywords = ['黃珊珊']
weeks=12
cond='or'
weeks=12
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
get_keyword_occurrence_time_series(df_query)


[{'x': '2022-01-29', 'y': 1},
 {'x': '2022-01-30', 'y': 0},
 {'x': '2022-01-31', 'y': 0},
 {'x': '2022-02-01', 'y': 0},
 {'x': '2022-02-02', 'y': 2},
 {'x': '2022-02-03', 'y': 1},
 {'x': '2022-02-04', 'y': 1},
 {'x': '2022-02-05', 'y': 3},
 {'x': '2022-02-06', 'y': 3},
 {'x': '2022-02-07', 'y': 3},
 {'x': '2022-02-08', 'y': 3},
 {'x': '2022-02-09', 'y': 1},
 {'x': '2022-02-10', 'y': 3},
 {'x': '2022-02-11', 'y': 1},
 {'x': '2022-02-12', 'y': 3},
 {'x': '2022-02-13', 'y': 0},
 {'x': '2022-02-14', 'y': 4},
 {'x': '2022-02-15', 'y': 0},
 {'x': '2022-02-16', 'y': 1},
 {'x': '2022-02-17', 'y': 4},
 {'x': '2022-02-18', 'y': 3},
 {'x': '2022-02-19', 'y': 0},
 {'x': '2022-02-20', 'y': 0},
 {'x': '2022-02-21', 'y': 1},
 {'x': '2022-02-22', 'y': 1},
 {'x': '2022-02-23', 'y': 0},
 {'x': '2022-02-24', 'y': 1},
 {'x': '2022-02-25', 'y': 2},
 {'x': '2022-02-26', 'y': 0},
 {'x': '2022-02-27', 'y': 1},
 {'x': '2022-02-28', 'y': 0},
 {'x': '2022-03-01', 'y': 0},
 {'x': '2022-03-02', 'y': 0},
 {'x': '20

In [ ]:
query_keywords = ['陳時中']
weeks=12
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
get_keyword_occurrence_time_series(df_query)


[{'x': '2022-01-29', 'y': 5},
 {'x': '2022-01-30', 'y': 5},
 {'x': '2022-01-31', 'y': 2},
 {'x': '2022-02-01', 'y': 0},
 {'x': '2022-02-02', 'y': 0},
 {'x': '2022-02-03', 'y': 0},
 {'x': '2022-02-04', 'y': 6},
 {'x': '2022-02-05', 'y': 13},
 {'x': '2022-02-06', 'y': 3},
 {'x': '2022-02-07', 'y': 11},
 {'x': '2022-02-08', 'y': 12},
 {'x': '2022-02-09', 'y': 13},
 {'x': '2022-02-10', 'y': 8},
 {'x': '2022-02-11', 'y': 10},
 {'x': '2022-02-12', 'y': 10},
 {'x': '2022-02-13', 'y': 3},
 {'x': '2022-02-14', 'y': 10},
 {'x': '2022-02-15', 'y': 15},
 {'x': '2022-02-16', 'y': 13},
 {'x': '2022-02-17', 'y': 13},
 {'x': '2022-02-18', 'y': 14},
 {'x': '2022-02-19', 'y': 10},
 {'x': '2022-02-20', 'y': 2},
 {'x': '2022-02-21', 'y': 11},
 {'x': '2022-02-22', 'y': 11},
 {'x': '2022-02-23', 'y': 10},
 {'x': '2022-02-24', 'y': 12},
 {'x': '2022-02-25', 'y': 6},
 {'x': '2022-02-26', 'y': 4},
 {'x': '2022-02-27', 'y': 1},
 {'x': '2022-02-28', 'y': 2},
 {'x': '2022-03-01', 'y': 2},
 {'x': '2022-03-02', 'y'

# Step 4: Process PK data

In [ ]:
# 定義PK對象
# 通常會多個關鍵字代表該PK對象，['柯文哲','柯p','柯P']
list_pkNames = ['陳時中', '黃珊珊', '蔣萬安']
list_pkKeywordSet = [['陳時中'], ['黃珊珊'], ['蔣萬安']]

# list_pkKeywordSet = [['陳時中'], ['黃珊珊'], ['柯文哲','柯p','柯P']]

# 線條顏色
list_colors = ["rgba(0,128,0,0.2)",'rgba(0,255,255,0.2)','rgba(0,0,255,0.2)']
#list_colors = ['green', 'red', 'blue']

# 人頭圖案
list_photos = [
    "https://upload.wikimedia.org/wikipedia/commons/c/c0/%E8%A1%9B%E7%94%9F%E7%A6%8F%E5%88%A9%E9%83%A8%E9%83%A8%E9%95%B7%E9%99%B3%E6%99%82%E4%B8%AD.jpg",
    "https://councilorwatch.tw/images/councillors/female/230.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/9/93/%E8%94%A3%E8%90%AC%E5%AE%89%E5%A7%94%E5%93%A1.jpg"
]

# 準備長條圖 線圖 總篇數 總次數等數據
list_freq_daily_line_chart = []
list_freq_news_category = []
list_sentimentInfo = []

list_total_articles=[]
list_total_frequency=[]

weeks = 12
cate = "全部"
cond = 'or'

selectedCategories = ['全部', '政治', '兩岸', '產經', '生活', '社會']
# selectedCategories = ['全部','政治', '產經', '生活', '國際', '社會', '兩岸']

for query_keywords in list_pkKeywordSet:

    # Filter news   
    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
    #df_query = filter_df_via_tokens(df, query_keywords, cond, cate, weeks)
    
    # Get line chart data
    line_xy_data = get_keyword_occurrence_time_series(df_query)
    list_freq_daily_line_chart.append(line_xy_data)
    
    # Get bar chart data, total articals and frequecy 
    cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(df_query, query_keywords)
    list_total_articles.append(total_articles)
    list_total_frequency.append(total_frequency)


    # We select these categories to display: ['全部', '政治', '兩岸', '產經', '生活', '社會']
    cate_freq_selected = [cate_occurrence[cate] for cate in selectedCategories]
    list_freq_news_category.append(cate_freq_selected)

    # Get sentiment information
    sentiCount, sentiPercnt = get_article_sentiment(df_query)
    #sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
    #                                            str(sentiPercnt['neg']))
    senti_info = [sentiPercnt[p] for p in ['pos','obj','neg']] # 只要取用pos,obj, neg的情緒數字篇數百分比
    list_sentimentInfo.append(senti_info)

# We need all the following data to display on our frontend page 有一大堆數據要送到前端去展示啊
pk_data =  {'list_freq_daily_line_chart': list_freq_daily_line_chart,
           'list_pkNames': list_pkNames,
           'list_colors': list_colors,
           'list_photos': list_photos,
           'list_freq_news_category': list_freq_news_category,
           'list_category': selectedCategories,
           'list_sentiInfo': list_sentimentInfo,
           'list_total_articles': list_total_articles,
           'list_total_frequency': list_total_frequency
           }

In [ ]:
pk_data

{'list_freq_daily_line_chart': [[{'x': '2022-01-29', 'y': 5},
   {'x': '2022-01-30', 'y': 5},
   {'x': '2022-01-31', 'y': 2},
   {'x': '2022-02-01', 'y': 0},
   {'x': '2022-02-02', 'y': 0},
   {'x': '2022-02-03', 'y': 0},
   {'x': '2022-02-04', 'y': 6},
   {'x': '2022-02-05', 'y': 13},
   {'x': '2022-02-06', 'y': 3},
   {'x': '2022-02-07', 'y': 11},
   {'x': '2022-02-08', 'y': 12},
   {'x': '2022-02-09', 'y': 13},
   {'x': '2022-02-10', 'y': 8},
   {'x': '2022-02-11', 'y': 10},
   {'x': '2022-02-12', 'y': 10},
   {'x': '2022-02-13', 'y': 3},
   {'x': '2022-02-14', 'y': 10},
   {'x': '2022-02-15', 'y': 15},
   {'x': '2022-02-16', 'y': 13},
   {'x': '2022-02-17', 'y': 13},
   {'x': '2022-02-18', 'y': 14},
   {'x': '2022-02-19', 'y': 10},
   {'x': '2022-02-20', 'y': 2},
   {'x': '2022-02-21', 'y': 11},
   {'x': '2022-02-22', 'y': 11},
   {'x': '2022-02-23', 'y': 10},
   {'x': '2022-02-24', 'y': 12},
   {'x': '2022-02-25', 'y': 6},
   {'x': '2022-02-26', 'y': 4},
   {'x': '2022-02-27', 'y'

# Step 5: Save PK data to csv file

In [ ]:
df_data_pk = pd.DataFrame(list(pk_data.items()),columns=['name','value'])

In [ ]:
df_data_pk

,name,value
0,list_freq_daily_line_chart,"[[{'x': '2022-01-29', 'y': 5}, {'x': '2022-01-..."
1,list_pkNames,"[陳時中, 黃珊珊, 蔣萬安]"
2,list_colors,"[rgba(0,128,0,0.2), rgba(0,255,255,0.2), rgba(..."
3,list_photos,[https://upload.wikimedia.org/wikipedia/common...
4,list_freq_news_category,"[[743, 110, 1, 8, 578, 1], [133, 44, 0, 2, 80,..."
5,list_category,"[全部, 政治, 兩岸, 產經, 生活, 社會]"
6,list_sentiInfo,"[[50, 20, 31], [49, 26, 26], [50, 20, 30]]"
7,list_total_articles,"[743, 133, 111]"
8,list_total_frequency,"[2628, 384, 453]"


In [ ]:
## 存成csv格式檔案
df_data_pk.to_csv('pk_taipei_mayor_election.csv',sep=',', index=None)

### Alternative way: using zip

In [ ]:
k=list(pk_data.keys())
v=list(pk_data.values())

In [ ]:
#list(zip(k,v))

In [ ]:
df_data_pk = pd.DataFrame(list(zip(k,v)),columns=['name','value'])
df_data_pk

,name,value
0,list_freq_daily_line_chart,"[[{'x': '2022-01-29', 'y': 5}, {'x': '2022-01-..."
1,list_pkNames,"[陳時中, 黃珊珊, 蔣萬安]"
2,list_colors,"[rgba(0,128,0,0.2), rgba(0,255,255,0.2), rgba(..."
3,list_photos,[https://upload.wikimedia.org/wikipedia/common...
4,list_freq_news_category,"[[743, 110, 1, 8, 578, 1], [133, 44, 0, 2, 80,..."
5,list_category,"[全部, 政治, 兩岸, 產經, 生活, 社會]"
6,list_sentiInfo,"[[49, 19, 30], [48, 25, 25], [50, 19, 29]]"
7,list_total_articles,"[743, 133, 111]"
8,list_total_frequency,"[2628, 384, 453]"


## Read csv file and convert to dict format

In [ ]:
df_data_pk = pd.read_csv('pk_taipei_mayor_election.csv')

In [ ]:
df_data_pk 

,name,value
0,list_freq_daily_line_chart,"[[{'x': '2022-01-29', 'y': 5}, {'x': '2022-01-..."
1,list_pkNames,"['陳時中', '黃珊珊', '蔣萬安']"
2,list_colors,"['rgba(0,128,0,0.2)', 'rgba(0,255,255,0.2)', '..."
3,list_photos,['https://upload.wikimedia.org/wikipedia/commo...
4,list_freq_news_category,"[[743, 110, 1, 8, 578, 1], [133, 44, 0, 2, 80,..."
5,list_category,"['全部', '政治', '兩岸', '產經', '生活', '社會']"
6,list_sentiInfo,"[[50, 20, 31], [49, 26, 26], [50, 20, 30]]"
7,list_total_articles,"[743, 133, 111]"
8,list_total_frequency,"[2628, 384, 453]"


In [ ]:
df_data_pk.values


array([['list_freq_daily_line_chart',
        "[[{'x': '2022-01-29', 'y': 5}, {'x': '2022-01-30', 'y': 5}, {'x': '2022-01-31', 'y': 2}, {'x': '2022-02-01', 'y': 0}, {'x': '2022-02-02', 'y': 0}, {'x': '2022-02-03', 'y': 0}, {'x': '2022-02-04', 'y': 6}, {'x': '2022-02-05', 'y': 13}, {'x': '2022-02-06', 'y': 3}, {'x': '2022-02-07', 'y': 11}, {'x': '2022-02-08', 'y': 12}, {'x': '2022-02-09', 'y': 13}, {'x': '2022-02-10', 'y': 8}, {'x': '2022-02-11', 'y': 10}, {'x': '2022-02-12', 'y': 10}, {'x': '2022-02-13', 'y': 3}, {'x': '2022-02-14', 'y': 10}, {'x': '2022-02-15', 'y': 15}, {'x': '2022-02-16', 'y': 13}, {'x': '2022-02-17', 'y': 13}, {'x': '2022-02-18', 'y': 14}, {'x': '2022-02-19', 'y': 10}, {'x': '2022-02-20', 'y': 2}, {'x': '2022-02-21', 'y': 11}, {'x': '2022-02-22', 'y': 11}, {'x': '2022-02-23', 'y': 10}, {'x': '2022-02-24', 'y': 12}, {'x': '2022-02-25', 'y': 6}, {'x': '2022-02-26', 'y': 4}, {'x': '2022-02-27', 'y': 1}, {'x': '2022-02-28', 'y': 2}, {'x': '2022-03-01', 'y': 2}, {'x': '

In [ ]:
# Convert to dictionary format
data = dict(list(df_data_pk.values))
data

{'list_freq_daily_line_chart': "[[{'x': '2022-01-29', 'y': 5}, {'x': '2022-01-30', 'y': 5}, {'x': '2022-01-31', 'y': 2}, {'x': '2022-02-01', 'y': 0}, {'x': '2022-02-02', 'y': 0}, {'x': '2022-02-03', 'y': 0}, {'x': '2022-02-04', 'y': 6}, {'x': '2022-02-05', 'y': 13}, {'x': '2022-02-06', 'y': 3}, {'x': '2022-02-07', 'y': 11}, {'x': '2022-02-08', 'y': 12}, {'x': '2022-02-09', 'y': 13}, {'x': '2022-02-10', 'y': 8}, {'x': '2022-02-11', 'y': 10}, {'x': '2022-02-12', 'y': 10}, {'x': '2022-02-13', 'y': 3}, {'x': '2022-02-14', 'y': 10}, {'x': '2022-02-15', 'y': 15}, {'x': '2022-02-16', 'y': 13}, {'x': '2022-02-17', 'y': 13}, {'x': '2022-02-18', 'y': 14}, {'x': '2022-02-19', 'y': 10}, {'x': '2022-02-20', 'y': 2}, {'x': '2022-02-21', 'y': 11}, {'x': '2022-02-22', 'y': 11}, {'x': '2022-02-23', 'y': 10}, {'x': '2022-02-24', 'y': 12}, {'x': '2022-02-25', 'y': 6}, {'x': '2022-02-26', 'y': 4}, {'x': '2022-02-27', 'y': 1}, {'x': '2022-02-28', 'y': 2}, {'x': '2022-03-01', 'y': 2}, {'x': '2022-03-02', 'y

## Convert to dictionary format (for reference)

In [ ]:
data={}
for k, v in df_data_pk.values:
    #print(eval(v))
    data[k]=eval(v)

In [ ]:
# Another approach to get the sampe result
data={}
for k,v in zip(df_data_pk.name, df_data_pk.value):
    data[k]=eval(v)
    

In [ ]:
data

{'list_freq_daily_line_chart': [[{'x': '2022-01-29', 'y': 5},
   {'x': '2022-01-30', 'y': 5},
   {'x': '2022-01-31', 'y': 2},
   {'x': '2022-02-01', 'y': 0},
   {'x': '2022-02-02', 'y': 0},
   {'x': '2022-02-03', 'y': 0},
   {'x': '2022-02-04', 'y': 6},
   {'x': '2022-02-05', 'y': 13},
   {'x': '2022-02-06', 'y': 3},
   {'x': '2022-02-07', 'y': 11},
   {'x': '2022-02-08', 'y': 12},
   {'x': '2022-02-09', 'y': 13},
   {'x': '2022-02-10', 'y': 8},
   {'x': '2022-02-11', 'y': 10},
   {'x': '2022-02-12', 'y': 10},
   {'x': '2022-02-13', 'y': 3},
   {'x': '2022-02-14', 'y': 10},
   {'x': '2022-02-15', 'y': 15},
   {'x': '2022-02-16', 'y': 13},
   {'x': '2022-02-17', 'y': 13},
   {'x': '2022-02-18', 'y': 14},
   {'x': '2022-02-19', 'y': 10},
   {'x': '2022-02-20', 'y': 2},
   {'x': '2022-02-21', 'y': 11},
   {'x': '2022-02-22', 'y': 11},
   {'x': '2022-02-23', 'y': 10},
   {'x': '2022-02-24', 'y': 12},
   {'x': '2022-02-25', 'y': 6},
   {'x': '2022-02-26', 'y': 4},
   {'x': '2022-02-27', 'y'

# views.py in Django

In [ ]:
from django.http import JsonResponse
import pandas as pd
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt

def load_data_pk():
    # Read data from csv file
    df_data_pk = pd.read_csv('app_taipei_mayor/dataset/pk_taipei_mayor_election.csv')
    
    global data
    #data={}
    # for k,v in zip(df_data_pk.name, df_data_pk.value):
    #     data[k]=eval(v)
    
    # Convert to dictionary format
    data = dict(list(df_data_pk.values)) # Just one line to convert them to dictionary


    # 沒用到的變數刪除之
    del df_data_pk

# load pk data
load_data_pk()

def home(request):
    return render(request,'app_taipei_mayor/home.html')

# csrf_exempt is used for POST
# 單獨指定這一支程式忽略csrf驗證
@csrf_exempt
def api_get_taipei_mayor_data(request):
    return JsonResponse(data)

print('Load app taipei mayor leaderboard...')


#  For reference: alternative aprroach to process response data with json format


In [ ]:
# 個別包含完整的資料
list_pkNames = ['陳時中', '黃珊珊', '蔣萬安']
list_pkKeywordSet = [['陳時中'], ['黃珊珊'], ['蔣萬安']]

# pkName = ['柯文哲', '蔡英文', '韓國瑜']
# pkset = [['柯文哲','柯p','柯P'], ['蔡英文'], ['韓國瑜']]

#list_colors = ['green', 'red', 'blue']
list_colors = ["rgba(0,128,0,0.2)",'rgba(0,255,255,0.2)','rgba(0,0,255,0.2)']

list_photos = [
    "https://upload.wikimedia.org/wikipedia/commons/c/c0/%E8%A1%9B%E7%94%9F%E7%A6%8F%E5%88%A9%E9%83%A8%E9%83%A8%E9%95%B7%E9%99%B3%E6%99%82%E4%B8%AD.jpg",
    "https://councilorwatch.tw/images/councillors/female/230.jpg",
    "https://upload.wikimedia.org/wikipedia/commons/9/93/%E8%94%A3%E8%90%AC%E5%AE%89%E5%A7%94%E5%93%A1.jpg"
]


def get_pk_data(name, query_keywords, photo, color):

    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)

    line_xy_data = get_keyword_occurrence_time_series(df_query)

    cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(
        df_query, query_keywords)
    selectedCategories = ['全部', '政治', '兩岸', '產經', '生活', '社會']
    # selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']
    # We select these categories to display: ['全部', '政治', '兩岸', '產經', '生活', '社會']
    freq_selected_categories = [cate_occurrence[k] for k in selectedCategories]

    # sentiment information
    sentiCount, sentiPercnt = get_article_sentiment(df_query)
    # sentiInfo = '正向:{}%, 中立:{}%, 負向:{}%'.format(str(sentiPercnt['pos']), str(sentiPercnt['obj']),
    #                                            str(sentiPercnt['neg']))
    sentiment = [sentiPercnt[p] for p in ['pos', 'obj', 'neg']]

    pk_data = {'line_xy_data': line_xy_data,
               'name': name,
               'color': color,
               'photo': photo,
               'freq_news_category': freq_selected_categories,
               'category': selectedCategories,
               'sentiment': sentiment,
               'total_articles': total_articles,
               'total_frequency': total_frequency,
               }
    return pk_data

pk_data_dict = {}
for i in range(len(list_pkNames)):
    name = list_pkNames[i]
    query_keywords = list_pkKeywordSet[i]
    photo = list_photos[i]
    color = list_colors[i]
    pk_data_dict[name] = get_pk_data(name, query_keywords, photo, color)



In [ ]:
pk_data_dict

{'陳時中': {'line_xy_data': [{'x': '2022-01-29', 'y': 5},
   {'x': '2022-01-30', 'y': 5},
   {'x': '2022-01-31', 'y': 2},
   {'x': '2022-02-01', 'y': 0},
   {'x': '2022-02-02', 'y': 0},
   {'x': '2022-02-03', 'y': 0},
   {'x': '2022-02-04', 'y': 6},
   {'x': '2022-02-05', 'y': 13},
   {'x': '2022-02-06', 'y': 3},
   {'x': '2022-02-07', 'y': 11},
   {'x': '2022-02-08', 'y': 12},
   {'x': '2022-02-09', 'y': 13},
   {'x': '2022-02-10', 'y': 8},
   {'x': '2022-02-11', 'y': 10},
   {'x': '2022-02-12', 'y': 10},
   {'x': '2022-02-13', 'y': 3},
   {'x': '2022-02-14', 'y': 10},
   {'x': '2022-02-15', 'y': 15},
   {'x': '2022-02-16', 'y': 13},
   {'x': '2022-02-17', 'y': 13},
   {'x': '2022-02-18', 'y': 14},
   {'x': '2022-02-19', 'y': 10},
   {'x': '2022-02-20', 'y': 2},
   {'x': '2022-02-21', 'y': 11},
   {'x': '2022-02-22', 'y': 11},
   {'x': '2022-02-23', 'y': 10},
   {'x': '2022-02-24', 'y': 12},
   {'x': '2022-02-25', 'y': 6},
   {'x': '2022-02-26', 'y': 4},
   {'x': '2022-02-27', 'y': 1},
 

### Save file using json format

In [ ]:

import json
# Serialize data into file:
json.dump( pk_data_dict, open( "pk_taipei_mayor_election.json", 'w' ) )

# Read data from file:
data = json.load( open( "pK_taipei_mayor_election.json" ) )
data

{'陳時中': {'line_xy_data': [{'x': '2022-01-29', 'y': 5},
   {'x': '2022-01-30', 'y': 5},
   {'x': '2022-01-31', 'y': 2},
   {'x': '2022-02-01', 'y': 0},
   {'x': '2022-02-02', 'y': 0},
   {'x': '2022-02-03', 'y': 0},
   {'x': '2022-02-04', 'y': 6},
   {'x': '2022-02-05', 'y': 13},
   {'x': '2022-02-06', 'y': 3},
   {'x': '2022-02-07', 'y': 11},
   {'x': '2022-02-08', 'y': 12},
   {'x': '2022-02-09', 'y': 13},
   {'x': '2022-02-10', 'y': 8},
   {'x': '2022-02-11', 'y': 10},
   {'x': '2022-02-12', 'y': 10},
   {'x': '2022-02-13', 'y': 3},
   {'x': '2022-02-14', 'y': 10},
   {'x': '2022-02-15', 'y': 15},
   {'x': '2022-02-16', 'y': 13},
   {'x': '2022-02-17', 'y': 13},
   {'x': '2022-02-18', 'y': 14},
   {'x': '2022-02-19', 'y': 10},
   {'x': '2022-02-20', 'y': 2},
   {'x': '2022-02-21', 'y': 11},
   {'x': '2022-02-22', 'y': 11},
   {'x': '2022-02-23', 'y': 10},
   {'x': '2022-02-24', 'y': 12},
   {'x': '2022-02-25', 'y': 6},
   {'x': '2022-02-26', 'y': 4},
   {'x': '2022-02-27', 'y': 1},
 